In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.0.0")
@file:DependsOn("io.ktor:ktor-client-cio:2.0.0")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.0.0")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.0.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe


In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import ro.jf.funds.user.sdk.UserSdk
import kotlinx.coroutines.runBlocking

val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"
val ACCOUNT_MATCHERS_CSV_FILE = "../../data/provision/account-matchers.csv"
val FUND_MATCHERS_CSV_FILE = "../../data/provision/fund-matchers.csv"
val EXCHANGE_MATCHERS_CSV_FILE = "../../data/provision/exchange-matchers.csv"
val LABEL_MATCHERS_CSV_FILE = "../../data/provision/label-matchers.csv"

val userSdk = UserSdk()
val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

val username = "Johann-7.1"
val user = runBlocking {
    userSdk.findUserByUsername(username)
        ?: userSdk.createUser(username)
}
user

UserTO(id=033d84b5-74df-43b1-8d6c-f39d0be30cf6, username=Johann-7.1)

In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
        .map { row -> CreateAccountTO(AccountName(row.getValue<String>("account")), Currency(row.getValue<String>("currency"))) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=59767459-fa88-4bd8-9ccf-404518c5b26c, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=6fa46eec-2fdd-413a-99fa-c184b9748d6b, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=8a9eee47-3997-4f3f-b585-55b56780d723, name=ING RON, unit=Currency(value=RON))
AccountTO(id=432ce797-a38f-4434-aa77-a3088850ee93, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=1a614ae3-1bb0-4110-9478-bd644e3d0e7a, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=2d1fe70e-f5a1-487b-9625-fc9d5ea015f2, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=9270e975-7030-4a90-b21e-02a04b360202, name=Other, unit=Currency(value=RON))
AccountTO(id=c83570a5-e091-460b-8e13-b9c6b7a77443, name=BT RON, unit=Currency(value=RON))
AccountTO(id=03f9b0cf-a331-4cf7-a3d2-99f36f3cf127, name=BT Economy RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
        .map { row -> CreateFundTO(FundName(row.getValue<String>("fund"))) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=245a9533-92a9-4d9b-a150-33edfc965431, name=Expenses)
FundTO(id=52a93dac-bc2d-4b07-8469-6ce3ae29485d, name=Work Income)
FundTO(id=1ee32a02-ddf4-47a3-9994-32a0ce0e7c40, name=Savings)

In [5]:
import ro.jf.funds.importer.api.model.AccountMatcherTO

val accountMatchers = runBlocking {
    DataFrame.readCSV(ACCOUNT_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row -> AccountMatcherTO(row.getValue<String>("import_account_name"), AccountName(row.getValue<String>("account_name"))) }
}
accountMatchers.map { it.toString() }.joinToString(separator = "\n")

AccountMatcherTO(importAccountName=ING old, accountName=ING RON)
AccountMatcherTO(importAccountName=ING Economy old, accountName=ING Economy)
AccountMatcherTO(importAccountName=Cash RON, accountName=Cash RON)
AccountMatcherTO(importAccountName=Euro, accountName=Cash EUR)
AccountMatcherTO(importAccountName=Food Coupons, accountName=Food Coupons)
AccountMatcherTO(importAccountName=Revolut - Manual, accountName=Revolut RON)
AccountMatcherTO(importAccountName=Other, accountName=Other)
AccountMatcherTO(importAccountName=BT RON, accountName=BT RON)
AccountMatcherTO(importAccountName=BT Economy RON, accountName=BT Economy RON)

In [6]:
import ro.jf.funds.importer.api.model.*

val fundMatchers = runBlocking {
    DataFrame.readCSV(FUND_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row ->
            when (row.getValue<String>("type")) {
                "by_account" -> FundMatcherTO.ByAccount(
                    importAccountName = row.getValue<String>("import_account_name"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )
                
                "by_label" -> FundMatcherTO.ByLabel(
                    importLabel = row.getValue<String>("import_label"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                "by_account_label" -> FundMatcherTO.ByAccountLabel(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                "by_account_label_with_transfer" -> FundMatcherTO.ByAccountLabelWithTransfer(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    initialFundName = FundName(row.getValue<String>("initial_fund_name")),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                else -> error("fund matcher type not recognized ${row.getValue<String>("type")}")
            }
        }
}
fundMatchers.map { it.toString() }.joinToString(separator = "\n")

ByAccountLabelWithTransfer(importAccountName=ING Economy old, importLabel=Invest Profit Dist, initialFundName=Savings, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=ING old, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=Food Coupons, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=BT RON, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=BT Economy RON, importLabel=Invest Profit Dist, initialFundName=Savings, fundName=Expenses)
ByLabel(importLabel=Basic - Food, fundName=Expenses)
ByLabel(importLabel=C&T - Gas & Parking, fundName=Expenses)
ByLabel(importLabel=Development - Education, fundName=Expenses)
ByLabel(importLabel=F&V - Fun, fundName=Expenses)
ByLabel(importLabel=S&S - Objects, fundName=Expenses)
ByLabel(importLabel=Home - House, fundName=Expenses)
ByLabe

In [7]:
val exchangeMatchers = runBlocking {
    DataFrame.readCSV(EXCHANGE_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row ->
            when (row.getValue<String>("type")) {
                "by_label" -> ExchangeMatcherTO.ByLabel(
                    label = row.getValue<String>("label"),
                )

                else -> error("exchange matcher type not recognized ${row.getValue<String>("type")}")
            }
        }
}
exchangeMatchers.map { it.toString() }.joinToString(separator = "\n")

ByLabel(label=Exchange)

In [8]:
val labelMatchers = runBlocking {
    DataFrame.readCSV(LABEL_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row ->
            LabelMatcherTO(row.getValue<String>("import_label"), Label(row.getValue<String>("label")))
        }
}
labelMatchers.map { it.toString() }.joinToString(separator = "\n")

LabelMatcherTO(importLabel=Basic - Food, label=basic)
LabelMatcherTO(importLabel=Home - House, label=home)
LabelMatcherTO(importLabel=C&T - Gas & Parking, label=transport)
LabelMatcherTO(importLabel=S&S - Objects, label=shopping_services)
LabelMatcherTO(importLabel=F&V - Fun, label=fun)
LabelMatcherTO(importLabel=Gifts, label=gifts)
LabelMatcherTO(importLabel=Development - Education, label=development)
LabelMatcherTO(importLabel=Work Income, label=income)
LabelMatcherTO(importLabel=Invest Profit Dist, label=profit_distributed)
LabelMatcherTO(importLabel=Exchange, label=exchange)

In [9]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles = File("../../data/wallet/2019")
    .listFiles()?.toList() ?: error("no files found")
val importConfiguration = ImportConfigurationTO(
    fileType = ImportFileTypeTO.WALLET_CSV,
    accountMatchers = accountMatchers,
    fundMatchers = fundMatchers,
    exchangeMatchers = exchangeMatchers,
    labelMatchers = labelMatchers
)
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=18490e4e-941a-4a82-9517-f9ab63f31e98, status=COMPLETED, reason=null)